FACILITY 

This is the Facility Data that will import all the mandatory columns in the (Asset-Facility Template)

In [17]:
import pandas as pd
import gradio as gr
import json
import os

# Load column mappings from a JSON file for flexibility
def load_column_mappings(config_file="column_mappings.json"):
    try:
        with open(config_file, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {
            "name*": "Building Name",
            "facilityType*": "Facility Type",
            "criticality": "Building Criticality",
            "location.longitude": "Longitude",
            "location.latitude": "Latitude",
            "location.location": "Building Address",
        }

# Normalize column headers
def normalize_headers(df):
    df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
    df.columns = df.columns.str.replace(r"\s+", " ", regex=True)  # Replace multiple spaces with one
    return df

# Function to validate and clean data
def validate_and_clean_data(df, required_columns, valid_criticality_values):
    # Normalize column headers
    df = normalize_headers(df)

    # Remove rows that are placeholders (e.g., "Mandatory")
    df = df[~df[required_columns["location.location"]].str.contains("Mandatory", case=False, na=False)]

    # Normalize criticality
    if required_columns["criticality"] in df:
        df[required_columns["criticality"]] = df[required_columns["criticality"]].str.extract(r"(C\d)")

    # Convert longitude and latitude to numeric
    if required_columns["location.longitude"] in df and required_columns["location.latitude"] in df:
        df[required_columns["location.longitude"]] = pd.to_numeric(df[required_columns["location.longitude"]], errors="coerce")
        df[required_columns["location.latitude"]] = pd.to_numeric(df[required_columns["location.latitude"]], errors="coerce")

    # Extract country and city from the address column
    if required_columns["location.location"] in df:
        # Split address into country and city
        df["location.country"] = df[required_columns["location.location"]].str.split(",").str[-1].str.strip()
        df["location.city"] = df[required_columns["location.location"]].str.split(",").str[-2].str.strip()

    # Filter valid rows
    df = df[
        df[required_columns["name*"]].notna() &
        (~df[required_columns["name*"]].str.contains("Mandatory|name", case=False, na=False)) &
        (~df[required_columns["facilityType*"]].str.contains("Mandatory|facility type", case=False, na=False)) &
        (df[required_columns["criticality"]].isin(valid_criticality_values))
    ]

    return df

# Main processing function
def process_data(afm_file, facility_template_file, namespace, output_path):
    try:
        # Load column mappings
        column_mappings = load_column_mappings()

        # Define valid criticality values
        valid_criticality_values = ["C1", "C2", "C3"]

        # Load the AFM data
        afm_data = pd.read_excel(afm_file.name, sheet_name="Building (Facility)", engine="openpyxl")

        # Normalize column headers
        afm_data = normalize_headers(afm_data)

        # Load the facility template
        facility_template_data = pd.read_csv(facility_template_file.name)

        # Validate and clean AFM data
        cleaned_afm_data = validate_and_clean_data(afm_data, column_mappings, valid_criticality_values)

        # Create new facility data
        facility_data = pd.DataFrame({
            "name*": cleaned_afm_data[column_mappings["name*"]],
            "facilityType*": cleaned_afm_data[column_mappings["facilityType*"]],
            "criticality": cleaned_afm_data[column_mappings["criticality"]],
            "location.longitude": cleaned_afm_data[column_mappings["location.longitude"]],
            "location.latitude": cleaned_afm_data[column_mappings["location.latitude"]],
            "location.location": cleaned_afm_data[column_mappings["location.location"]],
            "location.country": cleaned_afm_data["location.country"],
            "location.city": cleaned_afm_data["location.city"],
            "isActive*": True,
            "namespace*": namespace,
        })

        # Remove placeholder rows from the template
        facility_template_data = facility_template_data.drop(index=[0, 1], errors="ignore").reset_index(drop=True)

        # Merge the cleaned facility data with the template
        updated_facility_template = pd.concat([facility_template_data, facility_data], ignore_index=True)

        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Save the updated template
        updated_facility_template.to_csv(output_path, index=False)

        return f"Data processed successfully! Updated file saved at: {output_path}"
    except Exception as e:
        return f"An error occurred during processing: {str(e)}"

# Create the Gradio interface
interface = gr.Interface(
    fn=process_data,
    inputs=[
        gr.File(label="Upload Customer Template (.xlsx)"),
        gr.File(label="Upload Facility Template (.csv)"),
        gr.Textbox(label="Namespace*", placeholder="Enter namespace here"),
        gr.Textbox(label="Output Path", placeholder="Enter the full output file path"),
    ],
    outputs="text",
    title="FACILITY DATA ONBOARDING",
    description=(
        "Upload your AFM and Facility Template files, enter the namespace, specify the output file path, "
        "and process the data. The updated file will include separated country and city columns."
    )
)

# Launch the interface
interface.launch()



* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


C:\Users\remen\AppData\Local\Temp\ipykernel_8784\200874018.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[required_columns["criticality"]] = df[required_columns["criticality"]].str.extract(r"(C\d)")
C:\Users\remen\AppData\Local\Temp\ipykernel_8784\200874018.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[required_columns["location.longitude"]] = pd.to_numeric(df[required_columns["location.longitude"]], errors="coerce")
C:\Users\remen\AppData\Local\Temp\ipykernel_8784\200874018.py:42: Settin

After Importing (Asset-Facility Template) to system you will get ID's for Facilities which will be imported to (Asset-Location Template)

Location 

In [18]:
import pandas as pd
import gradio as gr
import warnings

# Suppress warnings
warnings.simplefilter("ignore", UserWarning)

# Define the function for data processing
def process_asset_location_data(asset_location_file, csv_file, output_path, namespace):
    try:
        # Validate that namespace is provided
        if not namespace.strip():
            return "Error: Namespace* must be provided."

        # Load the input files
        asset_location_data = pd.read_excel(asset_location_file.name, sheet_name='Asset,location')
        csv_data = pd.read_csv(csv_file.name)

        # Step 1: Clean the CSV data by removing the first row and placeholder values
        csv_data_cleaned = csv_data.iloc[1:]

        # Step 2: Extract unique 'Building', 'Floor', and 'Floor Criticality' data
        unique_building_floor_criticality = asset_location_data[['Building', 'Floor', 'Floor Criticality']].drop_duplicates().dropna()

        # Step 3: Filter out rows with placeholder values like 'Mandatory' or NaN
        valid_data = unique_building_floor_criticality[
            (unique_building_floor_criticality['Building'] != 'Mandatory') & 
            (unique_building_floor_criticality['Building'].notna()) & 
            (unique_building_floor_criticality['Floor'] != 'Mandatory') & 
            (unique_building_floor_criticality['Floor'].notna())
        ]

        # Step 4: Normalize floor values by removing extra spaces and standardizing formatting
        valid_data['Floor_normalized'] = valid_data['Floor'].str.replace(" ", "").str.replace("- ", "-")
        valid_data['Floor_normalized'] = valid_data['Floor_normalized'].str.replace("Core", "Core ", regex=False).str.replace("LVL", "LVL ", regex=False)

        # Step 5: Normalize and validate criticality values
        # Extract only valid criticality values (C1, C2, C3) using regex
        valid_data['Criticality'] = valid_data['Floor Criticality'].str.extract(r"(C\d)")
        
        # Define valid criticality values
        valid_criticality_values = ["C1", "C2", "C3"]
        
        # Filter rows based on valid criticality values
        valid_data = valid_data[valid_data['Criticality'].isin(valid_criticality_values)]

        # Step 6: Create a new DataFrame with cleaned building, normalized floor, and criticality data
        facility_name_column_cleaned = valid_data['Building'].reset_index(drop=True)
        name_column_cleaned = valid_data['Floor_normalized'].reset_index(drop=True)
        criticality_column_cleaned = valid_data['Criticality'].reset_index(drop=True)

        # Fill the `namespace*` column with the provided namespace value
        new_cleaned_data = pd.DataFrame({
            'facility*': [None] * len(facility_name_column_cleaned),
            'facility name': facility_name_column_cleaned,
            'name*': name_column_cleaned,
            'criticality': criticality_column_cleaned,
            'namespace*': [namespace] * len(facility_name_column_cleaned),
            'isActive*': [True] * len(facility_name_column_cleaned)  # Assuming active by default
        })

        # Step 7: Merge the new data with the original CSV template
        # Ensure only necessary columns are updated while keeping others as-is
        updated_csv_cleaned_data = csv_data_cleaned.copy()

        # Add or update the relevant columns in the existing template
        for col in ['facility*', 'facility name', 'name*', 'criticality', 'namespace*', 'isActive*']:
            if col in new_cleaned_data.columns:
                updated_csv_cleaned_data[col] = new_cleaned_data[col]

        # Save the cleaned data to the specified output file path
        updated_csv_cleaned_data.to_csv(output_path, index=False)

        return f"Data processed successfully! Updated file saved at: {output_path}"
    except Exception as e:
        return f"An error occurred during processing: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=process_asset_location_data,
    inputs=[
        gr.File(label="Upload Asset Location File (.xlsx)"),
        gr.File(label="Upload CSV File (.csv)"),
        gr.Textbox(label="Output File Path", placeholder="Enter namespace here"),
        gr.Textbox(label="Namespace*", placeholder="Enter the full output file path"),
    ],
    outputs="text",
    title="LOCATION DATA ONBOARDING",
    description=(
        "Upload your Asset Location and Facility Template files, specify the namespace, and process the data. "
        "Criticality values (C1, C2, C3) will be validated and included in the output."
    )
)

# Launch the interface
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7877

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


C:\Users\remen\AppData\Local\Temp\ipykernel_8784\555640678.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Floor_normalized'] = valid_data['Floor'].str.replace(" ", "").str.replace("- ", "-")
C:\Users\remen\AppData\Local\Temp\ipykernel_8784\555640678.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Floor_normalized'] = valid_data['Floor_normalized'].str.replace("Core", "Core ", regex=False).str.replace("LVL", "LVL ", regex=False)
C:\Users\remen\AppData\Local\Temp\ipykernel_8

This below code to map Facility id with location names 

In [19]:
import pandas as pd
import gradio as gr
import warnings

# Suppress warnings
warnings.simplefilter("ignore", UserWarning)

# Function to clean and truncate facility names automatically
def clean_and_truncate_facility_name(name, substrings_to_ignore):
    if isinstance(name, str):
        # Remove all unwanted substrings dynamically
        for substring in substrings_to_ignore:
            name = name.replace(substring, "")
        # Truncate to the first comma
        return name.split(",")[0].strip()
    return name

# Define the Gradio function
def process_facility_and_space_data(location_file, space_file, output_file_path, substrings_to_ignore):
    try:
        # Convert substrings to ignore from string to list
        substrings_to_ignore = substrings_to_ignore.split(',')

        # Load both files
        location_data = pd.read_csv(location_file.name)
        space_data = pd.read_csv(space_file.name)

        # Clean and truncate the 'name*' column in the location file for facility names
        location_data['name_cleaned'] = location_data['name*'].apply(
            lambda x: clean_and_truncate_facility_name(x, substrings_to_ignore)
        )

        # Clean and truncate the 'facility name' column in the space file
        space_data['facility_cleaned'] = space_data['facility name'].apply(
            lambda x: clean_and_truncate_facility_name(x, substrings_to_ignore)
        )

        # Create a mapping dictionary from the cleaned location file
        facility_mapping = location_data.set_index('name_cleaned')['id'].to_dict()

        # Populate the 'facility*' column in the space file based on the mapping
        space_data['facility*'] = space_data['facility_cleaned'].map(facility_mapping)

        # Drop temporary columns
        space_data = space_data.drop(columns=['facility_cleaned'])
        location_data = location_data.drop(columns=['name_cleaned'])

        # Save the updated space file
        space_data.to_csv(output_file_path, index=False)

        return f"Updated file saved at: {output_file_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=process_facility_and_space_data,
    inputs=[
        gr.File(label="Upload Exported Facility File (CSV)"),
        gr.File(label="Upload Location Template File (CSV)"),
        gr.Textbox(label="Output File Path", placeholder="Enter the full output file path"),
        gr.Textbox(
            label="Substrings to Ignore",
            placeholder="Enter substrings to ignore, separated by commas (e.g., Abu Dhabi,Al Ain,Alain)"
        ),
    ],
    outputs="text",
    title="EXPORTED FACILITY ID MAPPING IN LOCATION TEMPLATE",
    description=(
        "Upload your location and space CSV files, specify substrings to ignore, and the output file path. "
        "The tool will clean and truncate facility names and map IDs to save the updated CSV."
    )
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


SPACE CODE

In [20]:
import pandas as pd
import warnings
import gradio as gr

# Suppress warnings
warnings.simplefilter("ignore", UserWarning)

# Define a function to process the uploaded files
def process_asset_data(asset_location_file, csv_file, namespace, output_path):
    try:
        # Load data
        asset_location_data = pd.read_excel(asset_location_file.name, sheet_name='Asset,location', engine='openpyxl')
        csv_data = pd.read_csv(csv_file.name)

        # Step 1: Clean the CSV data by removing the first row and placeholder values
        csv_data_cleaned = csv_data.iloc[1:]

        # Step 2: Extract unique 'Building', 'Floor', 'Sublocation', and 'Criticality' data
        unique_building_floor = asset_location_data[['Building', 'Floor', 'Sublocation', 'Floor Criticality']].drop_duplicates().dropna()

        # Step 3: Filter out rows with placeholder values like 'Mandatory' or NaN
        valid_data = unique_building_floor[
            (unique_building_floor['Building'] != 'Mandatory') &
            (unique_building_floor['Building'].notna()) &
            (unique_building_floor['Floor'] != 'Mandatory') &
            (unique_building_floor['Floor'].notna()) &
            (unique_building_floor['Sublocation'] != 'Mandatory') &
            (unique_building_floor['Sublocation'].notna()) &
            (unique_building_floor['Floor Criticality'].notna())
        ]

        # Step 4: Normalize the `Sublocation` values (remove extra spaces and standardize formatting)
        valid_data['Sublocation_normalized'] = valid_data['Sublocation'].str.strip()
        valid_data['Sublocation_normalized'] = valid_data['Sublocation_normalized'].str.replace("  ", " ").str.replace("-", " - ")

        # Step 5: Normalize and validate criticality values
        # Extract valid criticality values (C1, C2, C3) using regex
        valid_data['Criticality'] = valid_data['Floor Criticality'].str.extract(r"(C\d)")
        
        # Define valid criticality values
        valid_criticality_values = ["C1", "C2", "C3"]
        
        # Filter rows based on valid criticality values
        valid_data = valid_data[valid_data['Criticality'].isin(valid_criticality_values)]

        # Extract unique normalized values
        valid_data = valid_data.drop_duplicates(subset=['Sublocation_normalized'])

        # Step 6: Create a new DataFrame with cleaned building, floor, sublocation, and criticality data
        facility_name_column_cleaned = valid_data['Building'].reset_index(drop=True)
        location_name_column_cleaned = valid_data['Floor'].reset_index(drop=True)
        name_column_cleaned = valid_data['Sublocation_normalized'].reset_index(drop=True)
        criticality_column_cleaned = valid_data['Criticality'].reset_index(drop=True)

        # Fill the `namespace*` column with the user-provided namespace
        new_cleaned_data = pd.DataFrame({
            'facility name': facility_name_column_cleaned,
            'location name': location_name_column_cleaned,
            'name*': name_column_cleaned,
            'criticality': criticality_column_cleaned,
            'namespace*': [namespace] * len(facility_name_column_cleaned),
            'isActive*': [True] * len(facility_name_column_cleaned)  # Assuming active by default
        })

        # Step 7: Concatenate the cleaned new data with the original cleaned CSV data
        updated_csv_cleaned_data = pd.concat([csv_data_cleaned, new_cleaned_data], ignore_index=True)

        # Save the cleaned data to the specified output path
        updated_csv_cleaned_data.to_csv(output_path, index=False)

        return f"Cleaned data has been successfully saved to: {output_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=process_asset_data,
    inputs=[
        gr.File(label="Upload Asset Location File (.xlsx)"),
        gr.File(label="Upload Space Template File (.csv)"),
        gr.Textbox(label="Namespace*", placeholder="Enter namespace here"),
        gr.Textbox(label="Output Path", placeholder="Enter the full output file path"),
    ],
    outputs="text",
    title="SPACE DATA ONBOARDING",
    description=(
        "Upload your Asset Location and CSV files, enter the namespace, specify the output file path, "
        "and process the data. Criticality values (C1, C2, C3) will be validated and included in the output."
    )
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


C:\Users\remen\AppData\Local\Temp\ipykernel_8784\932199211.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Sublocation_normalized'] = valid_data['Sublocation'].str.strip()
C:\Users\remen\AppData\Local\Temp\ipykernel_8784\932199211.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Sublocation_normalized'] = valid_data['Sublocation_normalized'].str.replace("  ", " ").str.replace("-", " - ")
C:\Users\remen\AppData\Local\Temp\ipykernel_8784\932199211.py:38: SettingWithCopyWarning

This below code to map Location id with Space Names

In [21]:
import pandas as pd
import gradio as gr

# Function to clean and normalize facility names
def clean_and_normalize_facility_name(name):
    if isinstance(name, str):
        # Normalize spaces around dashes and remove leading/trailing spaces
        name = name.replace("- ", "-").replace(" -", "-").strip()
        # Truncate to the first comma
        return name.split(",")[0].strip()
    return name

# Function to process the data
def process_facility_data(location_file, space_file, output_path):
    try:
        # Load the uploaded files
        location_data = pd.read_csv(location_file.name)
        space_data = pd.read_csv(space_file.name)

        # Clean and normalize facility names in the location dataset
        location_data['name_cleaned'] = location_data['name*'].apply(clean_and_normalize_facility_name)

        # Clean and normalize location names in the space dataset
        space_data_temp = space_data.copy()
        space_data_temp['location_cleaned'] = space_data_temp['location name'].astype(str).apply(clean_and_normalize_facility_name)

        # Create a mapping dictionary using the cleaned location file
        if 'facility name' not in location_data.columns or 'name_cleaned' not in location_data.columns or 'id' not in location_data.columns:
            return "Error: Missing required columns ('facility name', 'name*', or 'id') in the location file."
        
        location_mapping = location_data.set_index(['facility name', 'name_cleaned'])['id'].to_dict()

        # Populate the "location*" column in the space file using the mapping
        if 'facility name' not in space_data.columns or 'location name' not in space_data.columns:
            return "Error: Missing required columns ('facility name' or 'location name') in the space file."

        space_data['location*'] = space_data_temp.apply(
            lambda row: location_mapping.get((row['facility name'], row['location_cleaned']), None),
            axis=1
        )

        # Save the updated space file to the specified output path
        if not output_path.strip():
            return "Error: Output path cannot be empty."
        
        space_data.to_csv(output_path, index=False)

        return f"Data processed successfully! Updated file saved at: {output_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=process_facility_data,
    inputs=[
        gr.File(label="Upload EXPORTED LOCATION FILE (.csv)"),
        gr.File(label="Upload Space File (.csv)"),
        gr.Textbox(label="Output Path", placeholder="Enter the full output file path"),
    ],
    outputs="text",
    title="LOCATION ID MAPPING WITH SPACE TEMPLATE",
    description=(
        "Upload your Location and Space CSV files, specify the output file path, "
        "and process the data. The updated file will overwrite the specified Space file."
    )
)

# Launch the interface
interface.launch()


* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


EQUIPMENT

In [13]:
import pandas as pd
import gradio as gr

# Function to process the data
def process_asset_equipment_data(asset_location_file, csv_file, namespace, output_path):
    try:
        # Load data from uploaded files
        asset_location_data = pd.read_excel(asset_location_file.name, sheet_name='Asset,location (2)', engine='openpyxl')
        csv_data = pd.read_csv(csv_file.name)

        # Step 1: Clean the CSV data by removing the first row
        csv_data_cleaned = csv_data.iloc[1:]

        # Step 2: Extract unique data from the "Asset,location" sheet
        unique_asset_data = asset_location_data[
            ['Barcode', 'Asset System', 'Asset / Equipment', 'Asset Criticality', 'Sublocation']
        ].drop_duplicates()

        # Step 3: Normalize criticality values
        unique_asset_data['Asset Criticality'] = unique_asset_data['Asset Criticality'].str.extract(r'^(C[1-3])')

        # Step 4: Filter valid rows
        placeholder_values = ['Mandatory', None, '']
        valid_data_normalized = unique_asset_data[
            (~unique_asset_data['Barcode'].isin(placeholder_values)) &
            (~unique_asset_data['Asset System'].isin(placeholder_values)) &
            (~unique_asset_data['Asset / Equipment'].isin(placeholder_values)) &
            (~unique_asset_data['Sublocation'].isin(placeholder_values)) &
            (unique_asset_data['Asset Criticality'].isin(['C1', 'C2', 'C3']))
        ]

        # Step 5: Populate required columns
        barcode_column = valid_data_normalized['Barcode'].reset_index(drop=True)
        class_column = valid_data_normalized['Asset System'].reset_index(drop=True)
        equipment_type_column = valid_data_normalized['Asset / Equipment'].reset_index(drop=True)
        criticality_column = valid_data_normalized['Asset Criticality'].reset_index(drop=True)
        asset_name_column = valid_data_normalized['Sublocation'].reset_index(drop=True)

        # Fill namespace and other template columns
        new_cleaned_data_normalized = pd.DataFrame({
            'name*': barcode_column,
            'class*': class_column,
            'equipmentType*': equipment_type_column,
            'criticality': criticality_column,
            'asset name': asset_name_column,
            'namespace*': [namespace] * len(barcode_column),
            'isActive*': [True] * len(barcode_column)
        })

        # Align with template columns
        template_columns = csv_data_cleaned.columns
        for column in template_columns:
            if column not in new_cleaned_data_normalized.columns:
                new_cleaned_data_normalized[column] = None

        new_cleaned_data_normalized = new_cleaned_data_normalized[template_columns]

        # Concatenate with cleaned CSV data
        updated_csv_cleaned_data_final = pd.concat([csv_data_cleaned, new_cleaned_data_normalized], ignore_index=True)

        # Save to the specified output path
        updated_csv_cleaned_data_final.to_csv(output_path, index=False)

        return f"Cleaned data has been successfully saved to: {output_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=process_asset_equipment_data,
    inputs=[
        gr.File(label="Upload Asset Location File (.xlsx)"),
        gr.File(label="Upload CSV File (.csv)"),
        gr.Textbox(label="Namespace*", placeholder="Enter namespace here"),
        gr.Textbox(label="Output Path", placeholder="Enter the full output file path"),
    ],
    outputs="text",
    title="Asset Equipment Data Processor",
    description=(
        "Upload your Asset Location and Asset Equipment CSV files, provide a namespace, "
        "specify the output file path, and process the data. The cleaned data will be saved to the specified path."
    )
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


c:\Users\remen\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Asset Id's Update

In [14]:
import pandas as pd
import gradio as gr

# Function to process the data
def process_space_and_location_data(location_file, space_file, output_path):
    try:
        # Load the uploaded files
        location_data = pd.read_csv(location_file.name)
        space_data = pd.read_csv(space_file.name)

        # Clean the relevant columns
        location_data['name_cleaned'] = location_data['name*'].str.strip()
        space_data['asset_name_cleaned'] = space_data['asset name'].astype(str).str.strip()

        # Create a mapping dictionary using the location file
        location_mapping = location_data.set_index('name_cleaned')['id'].to_dict()

        # Populate the "asset*" column in the space file based on asset name match
        space_data['asset*'] = space_data['asset_name_cleaned'].map(location_mapping)

        # Drop the helper column to keep the template unchanged
        space_data.drop(columns=['asset_name_cleaned'], inplace=True)

        # Save the updated space file to the specified output path
        space_data.to_csv(output_path, index=False)

        return f"Data processed successfully! Updated file saved at: {output_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=process_space_and_location_data,
    inputs=[
        gr.File(label="Upload Location File (.csv)"),
        gr.File(label="Upload Space File (.csv)"),
        gr.Textbox(label="Output Path", placeholder="Enter the full output file path"),
    ],
    outputs="text",
    title="Space and Location Data Mapper",
    description=(
        "Upload your Location and Space CSV files, specify the output file path, "
        "and process the data. The updated file will overwrite the specified Space file."
    )
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Barcode Mapping

In [11]:
import pandas as pd

# Load the Excel file
file_path = 'A:\\Alfuttaim Onboardings\\Doha Onboarding\\Copy of Contract and asset data template Issue- IKEA.xlsx'
output_file_path = 'A:\\Alfuttaim Onboardings\\Doha Onboarding\\Updated_Asset_Barcodes6.xlsx'

# Load the 'Asset,location' sheet
excel_data = pd.ExcelFile(file_path)
asset_location_df = excel_data.parse('Asset,location')

# Set 'Building' column to 'IKEA' for all rows
asset_location_df['Building']

# Fill missing values in other relevant columns with placeholders
asset_location_df['Floor'] = asset_location_df['Floor'].fillna('UnknownFloor')
asset_location_df['Sublocation'] = asset_location_df['Sublocation'].fillna('UnknownSublocation')
asset_location_df['Asset / Equipment'] = asset_location_df['Asset / Equipment'].fillna('UnknownAsset')

# Identify duplicates and add a counter only for duplicate combinations of Facility, Floor, Sublocation, and Asset / Equipment
duplicates_mask = asset_location_df.duplicated(subset=['Building', 'Floor', 'Sublocation', 'Asset / Equipment'], keep=False)
asset_location_df['Counter'] = 0
asset_location_df.loc[duplicates_mask, 'Counter'] = asset_location_df.groupby([
    'Building', 'Floor', 'Sublocation', 'Asset / Equipment'
]).cumcount() + 1

# Create the Barcode column with numbering only for duplicates
asset_location_df['Barcode'] = (
    asset_location_df['Building'] + "_" +
    asset_location_df['Floor'].str.replace(' ', '_') + "_" +
    asset_location_df['Sublocation'].str.replace(' ', '_') + "_" +
    asset_location_df['Asset / Equipment'].str.replace(' ', '_') +
    asset_location_df['Counter'].apply(lambda x: f"_{x}" if x > 0 else "")
)

# Drop the Counter column as it's no longer needed
asset_location_df = asset_location_df.drop(columns=['Counter'])

# Save the updated data to a new Excel file
asset_location_df.to_excel(output_file_path, sheet_name='Asset,location', index=False)

print(f"The updated file with the 'Barcode' column has been saved to: {output_file_path}")



c:\Users\remen\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


The updated file with the 'Barcode' column has been saved to: A:\Alfuttaim Onboardings\Doha Onboarding\Updated_Asset_Barcodes6.xlsx
